## Contador de Objetos
##### por JR Blanco

- Asignatura: Modelos de Inteligencia Artificial

- ## [Enlace video](https://educantabria-my.sharepoint.com/:v:/g/personal/jblancog03_educantabria_es/EayeL3u8qSBGh1JOIt-ibz8BE6dEmHhrmCICxD3K0TTKZA?nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJPbmVEcml2ZUZvckJ1c2luZXNzIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXciLCJyZWZlcnJhbFZpZXciOiJNeUZpbGVzTGlua0NvcHkifX0&email=adiezc09%40educantabria.es&e=4JUCGT)


In [1]:
#Cargas las librerias necesarias
from ultralytics import YOLO
import cv2
import datetime

Cargamos el modelo Yolo Nano

In [2]:
model = YOLO('yolov8m.pt')
model.info()

100%|██████████| 49.7M/49.7M [00:02<00:00, 20.6MB/s]


YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs


(295, 25902640, 0, 79.3204224)

### Clase Objeto_Contar

Decidí desarrollar mi propia clase para contar vehículos al cruzar una línea específica, por las deficiencias que encontré en las bibliotecas proporcionadas por ultralytics.solutions. Estas bibliotecas no lograban identificar de manera precisa todos los vehículos que pasaban, resultando en una cantidad significativa de falsos negativos. Esto derivó en una tasa de éxito apenas superior al 40%, un rendimiento muy por debajo de lo esperado. 

Tras un análisis detallado del objeto devuelto por el método .track, descubrí que 'boxes' es una lista que contiene los objetos detectados. Este hallazgo me impulsó a desarrollar mi propio detector, logrando una asombrosa tasa de acierto cercana al 99% con el material de video que utilicé para las pruebas. Este nuevo enfoque no solo superó ampliamente las limitaciones previas, sino que también estableció un precedente en la eficacia de la detección y conteo de vehículos en mi proyecto.

In [3]:
class Objeto_Contar:
    """
    Clase para contar objetos en una línea específica de un frame de video.
    """
    
    def __init__(self, linea, show_linea=True, color_linea=(0, 255, 0), color_texto=(0, 0, 0)):
        """
        Inicializa la clase Objeto_Contar.

        Parámetros:
        - linea: tupla, coordenadas de inicio y fin de la línea de conteo.
        - show_linea: bool, indica si se debe mostrar la línea de conteo en el frame.
        - color_linea: tupla, color de la línea de conteo en formato BGR.
        - color_texto: tupla, color del texto de conteo en formato BGR.
        """
        self.cuenta = 0
        self.inicio_linea = linea[0]
        self.fin_linea = linea[1]
        self.show_linea = show_linea
        self.color_linea = color_linea
        self.color_texto = color_texto
        self.__dicionario = {} #Para no contar el mismo objeto(id) dos veces o más
        
    def contar(self, frame, tracker):
        """
        Cuenta los objetos que cruzan la línea de conteo en el frame.

        Parámetros:
        - frame: numpy.ndarray, frame de video.
        - tracker: ultralytics.solutions.tracker.Tracker, objeto de seguimiento de objetos, se obtiene en el método detectar de la clase YOLO.

        """
        for caja in tracker[0].boxes:
            id = int(caja.id)
            x, y = caja.xywh[0][0], caja.xywh[0][1]
            
            if x > self.inicio_linea[0] and x < self.fin_linea[0] and y > self.inicio_linea[1]:
                if id not in self.__dicionario:
                    self.__dicionario[id] = True
                    self.cuenta += 1           
        
        if self.show_linea:
            cv2.line(frame, self.inicio_linea, self.fin_linea, self.color_linea, 1)
            cv2.rectangle(frame, (self.inicio_linea[0], self.inicio_linea[1]-13), (self.inicio_linea[0]+30, self.inicio_linea[1]), self.color_linea, -1)
            #Para mostrar el texto de la cuenta en el centro del rectangulo
            text = f"{self.cuenta}"
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)
            text_x = self.inicio_linea[0] + (30 - text_size[0]) // 2
            text_y = self.inicio_linea[1] - 2
            cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, self.color_texto, 1)
            

Cargar el Video

In [4]:
video = './trafficCam.mp4'

Definine las lineas para usar de contadores

In [5]:
#Lineas para el conteo de objetos
lineaA = ((40, 480), (500, 480))
lineaB = ((525, 480), (745, 480))
lineaC = ((850, 480), (1190, 480))

### Main - Principal

In [6]:
captura = cv2.VideoCapture(video)

#Objetos para contar, uno por cada línea
obj_contador_A = Objeto_Contar(linea=lineaA, color_linea=(0, 255, 0))
obj_contador_B = Objeto_Contar(linea=lineaB, color_linea=(255, 255, 0))
obj_contador_C = Objeto_Contar(linea=lineaC, color_linea=(0, 255, 255))

while True:
    start = datetime.datetime.now() #Contador de tiempo para los FPS - Tiempos Inicial
    
    ret, frame = captura.read() #Lee cada frame del video
    if not ret:
        break
    
    resultado = model.track(frame, persist=True) #Detecta los objetos y hace el track
    
    #Metodos de la clase Objeto_Contar que cuenta si los objetos pasan la linea
    obj_contador_A.contar(frame, resultado) 
    obj_contador_B.contar(frame, resultado) 
    obj_contador_C.contar(frame, resultado) 
    
    end = datetime.datetime.now() #Contador de tiempo para los FPS - Tiempo Final
    
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}" #Calculo los FPS
    cv2.putText(frame, fps, (10, 25), 1, 1, (0, 0, 255), 2) #Muestro en pantalla los FPS
    
    cv2.imshow('Video', resultado[0].plot(line_width=1)) #Muestro la visualización del resultado del tracks -> Dibuja la dDtección de los vehiculos
    #cv2.imshow('Video', frame)
    if cv2.waitKey(1) == 27:   #Metodo para salir del bucle al pulsa ESC
        break

captura.release()
cv2.destroyAllWindows()

#Muestra el resultado al finalizar el video
print("\nConteo de vehiculos por linea")
print(f"Total de vehiculos que pasaron por la linea A: {obj_contador_A.cuenta}")
print(f"Total de vehiculos que pasaron por la linea B: {obj_contador_B.cuenta}")
print(f"Total de vehiculos que pasaron por la linea C: {obj_contador_C.cuenta}")


0: 384x640 7 cars, 73.2ms
Speed: 4.0ms preprocess, 73.2ms inference, 361.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 32.5ms
Speed: 4.2ms preprocess, 32.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 33.2ms
Speed: 2.5ms preprocess, 33.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 34.0ms
Speed: 1.7ms preprocess, 34.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 33.8ms
Speed: 1.7ms preprocess, 33.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 29.4ms
Speed: 2.3ms preprocess, 29.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 29.1ms
Speed: 2.4ms preprocess, 29.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 29.5ms
Speed: 4.1ms preprocess, 29.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 38